# COGS 108 - Data Checkpoint

# Names

- Jamie Wei
- Alexis Garduno
- James Daza
- Aleksander Archipov

<a id='research_question'></a>
# Research Question

- Does the number of times a player is traded predict that player's performance (e.g. average total points scored/game, average minutes played/game, etc) in an NBA game? Also, does the turn-over rate (e.g. number of players traded within a team/season) of a NBA team affect the likelihood that the team will reach the NBA finals (evidenced by the last 20 years of NBA games)?

# Dataset(s)

### Source: NBA_ABI 
- Link to the dataset: https://github.com/swar/nba_api
- Number of observations: N/A
- API Client that allows access to various NBA's stats API endpoints

### Dataset Name: NBA Stats (replace with boxscore and team performance datasets)
- Link to the dataset: https://www.nba.com/stats/
- Number of observations: 12
- Player stats from 10 random teams ranging from 2016-2021


# Setup

In [1]:
import pandas as pd
import numpy as np
import time
import warnings
warnings.filterwarnings(action='once')

from nba_api.stats.static import players
from nba_api.stats.endpoints import playercareerstats

### Gathering Teams

In [2]:
#Import the roster of teams from the NBA API
from nba_api.stats.static import teams

nba_teams = teams.get_teams()

We select 10 nba teams at random 

In [3]:
#Obtain a full list of all abbreviations - will need abbreviations to identify team statistics
nba_teams_df=pd.DataFrame(nba_teams)
team_id=nba_teams_df['id'] #this is the unique team id
team_id_random=np.random.choice(team_id,10,replace=False) #identify ten team ids
nba_teams_rdf=nba_teams_df[nba_teams_df['id'].isin(team_id_random)] #df of 10 randomly selected teams
nba_teams_rdf

,id,full_name,abbreviation,nickname,city,state,year_founded
1,1610612738,Boston Celtics,BOS,Celtics,Boston,Massachusetts,1946
4,1610612741,Chicago Bulls,CHI,Bulls,Chicago,Illinois,1966
5,1610612742,Dallas Mavericks,DAL,Mavericks,Dallas,Texas,1980
8,1610612745,Houston Rockets,HOU,Rockets,Houston,Texas,1967
15,1610612752,New York Knicks,NYK,Knicks,New York,New York,1946
19,1610612756,Phoenix Suns,PHX,Suns,Phoenix,Arizona,1968
20,1610612757,Portland Trail Blazers,POR,Trail Blazers,Portland,Oregon,1970
23,1610612760,Oklahoma City Thunder,OKC,Thunder,Oklahoma City,Oklahoma,1967
27,1610612764,Washington Wizards,WAS,Wizards,Washington,District of Columbia,1961
28,1610612765,Detroit Pistons,DET,Pistons,Detroit,Michigan,1948


### Requesting Games
Now we request the games for all of the ten teams we have selected

In [4]:
#Pull all games for all ten teams

#Documentation for this endpoint: 
#https://github.com/swar/nba_api/blob/master/docs/nba_api/stats/endpoints/leaguegamefinder.md
from nba_api.stats.endpoints import leaguegamefinder

# Query for games from the League Game Finder
gamefinder=pd.DataFrame()
for i in team_id_random:
    time.sleep(1) #delay to prevent being blocked from the API
    df = leaguegamefinder.LeagueGameFinder(team_id_nullable=[i]).get_data_frames()[0] #parameter of team ids given
    gamefinder = pd.concat([df,gamefinder])

In [5]:
#Game Statistics

#One row corresponds to one game and one team.
#There will be two rows per game, since there are two teams that played each other.
#Will need to exclude duplicate rows (XXXX will remove duplicate rows)
print(list(set(gamefinder.TEAM_ID))) #confirmed that identified 10 different teams
print(gamefinder.shape) #31,386 games

##Game Finder Dataset: This dataset will be used as the outcome when we look at the association between 
##the exposure and outcome relationship. 
gamefinder

[1610612738, 1610612741, 1610612742, 1610612745, 1610612752, 1610612756, 1610612757, 1610612760, 1610612764, 1610612765]
(35246, 28)


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22021,1610612764,WAS,Washington Wizards,0022100464,2022-02-17,WAS @ BKN,W,240,117,...,0.813,8,30,38,26,10,7,15,21,14.0
1,22021,1610612764,WAS,Washington Wizards,0022100876,2022-02-16,WAS @ IND,L,240,108,...,0.813,12,32,44,25,7,3,12,20,-5.0
2,22021,1610612764,WAS,Washington Wizards,0022100858,2022-02-14,WAS vs. DET,W,241,103,...,0.778,10,35,45,21,8,3,15,16,9.0
3,22021,1610612764,WAS,Washington Wizards,0022100847,2022-02-12,WAS vs. SAC,L,240,110,...,0.714,10,22,32,26,11,3,9,19,-13.0
4,22021,1610612764,WAS,Washington Wizards,0022100832,2022-02-10,WAS vs. BKN,W,241,113,...,0.889,10,31,41,29,7,5,11,23,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3640,21983,1610612738,BOS,Boston Celtics,0028300052,1983-11-05,BOS @ WAS,W,240,120,...,0.778,5,28,33,36,7,7,16,26,NaN
3641,21983,1610612738,BOS,Boston Celtics,0028300043,1983-11-04,BOS vs. IND,W,240,121,...,0.841,11,34,45,28,8,5,22,26,NaN
3642,21983,1610612738,BOS,Boston Celtics,0028300033,1983-11-02,BOS vs. MIL,W,240,119,...,0.816,14,24,38,26,7,7,18,18,NaN
3643,21983,1610612738,BOS,Boston Celtics,0028300019,1983-10-29,BOS @ CLE,W,240,108,...,0.690,13,31,44,25,12,7,13,22,NaN


Now we request all the player stats from seasons ranging from 2016-2021

In [6]:
#get game ids for the last five years

#for now, let's focus on the last five seasons for ease
from nba_api.stats.endpoints import playergamelogs

#generate a parameter dataframe to define timeframe
#Is this timeframe correct? What is the timeframe that the season normally runs from?
season_parameter_df=pd.DataFrame({'Season':['2016-17','2017-18','2018-19','2019-20','2020-21'], 
                    'Date_From':['9/01/2016','9/01/2017','9/01/2018','9/01/2019','9/01/2020'],
                    'Date_To':['8/31/2017','8/31/2018','8/31/2019','8/31/2020','8/31/2021']})

#will obtain no game ids, without the season_nullable and date_nullable items selected
logsdf=pd.DataFrame()
for i in list(range(1, 5)):
    time.sleep(1)
    season=season_parameter_df.iloc[i][0]
    date_from=season_parameter_df.iloc[i][1]
    date_to=season_parameter_df.iloc[i][2]
    
    logs = pd.DataFrame(playergamelogs.PlayerGameLogs(
        season_nullable = season,
        date_from_nullable = date_from,                                                     
        date_to_nullable = date_to
    ).player_game_logs.get_data_frame())
    logsdf = pd.concat([logs,logsdf])

In [7]:
#unique set of game ids
game_ids = list(set(logsdf['GAME_ID'])) #not sure if we need game_ids
player_ids = pd.DataFrame(list(set(logsdf['PLAYER_ID'])))
print(pd.DataFrame(game_ids).shape) #4599 game ids
print(player_ids.shape) #875 unique players
print(logsdf.shape) #97,655 rows where each row is for each player in each game. 

(4599, 1)
(875, 1)
(97655, 66)


<b>4599 Games <br>
875 Unique Players <br>
97,655 stats of players in each game<b>

All the columns of every player that played in the seasons we selected from the random teams chosen

In [8]:
#preview, likely not use these columns
logsdf.columns

Index(['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'NICKNAME', 'TEAM_ID',
       'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID', 'GAME_DATE', 'MATCHUP',
       'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM',
       'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK',
       'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS', 'NBA_FANTASY_PTS', 'DD2',
       'TD3', 'GP_RANK', 'W_RANK', 'L_RANK', 'W_PCT_RANK', 'MIN_RANK',
       'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK', 'FG3M_RANK', 'FG3A_RANK',
       'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK', 'FT_PCT_RANK', 'OREB_RANK',
       'DREB_RANK', 'REB_RANK', 'AST_RANK', 'TOV_RANK', 'STL_RANK', 'BLK_RANK',
       'BLKA_RANK', 'PF_RANK', 'PFD_RANK', 'PTS_RANK', 'PLUS_MINUS_RANK',
       'NBA_FANTASY_PTS_RANK', 'DD2_RANK', 'TD3_RANK', 'VIDEO_AVAILABLE_FLAG'],
      dtype='object')

# Data Cleaning

### Cleaning Player Logs

In [9]:
# Copy Logs DataFrame and remove TEAM ID's that were not selected
logsDf = logsdf.copy()


# Restrict to the player stats we want
logsDf = logsDf[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'MIN', 'FG_PCT', 'FT_PCT', 'PTS', 'AST', 'REB', 'STL', 'BLK', 'PLUS_MINUS']]
logsDf


,SEASON_YEAR,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,MIN,FG_PCT,FT_PCT,PTS,AST,REB,STL,BLK,PLUS_MINUS
0,2020-21,203469,Cody Zeller,1610612766,Charlotte Hornets,29.645000,0.500,0.833,11,2,4,0,0,-2
1,2020-21,201144,Mike Conley,1610612762,Utah Jazz,21.833333,0.400,0.000,11,9,6,3,0,28
2,2020-21,1626195,Willy Hernangomez,1610612740,New Orleans Pelicans,29.578333,0.727,0.600,19,1,13,0,2,-16
3,2020-21,1629605,Tacko Fall,1610612738,Boston Celtics,16.586667,0.500,0.500,3,0,8,0,3,-6
4,2020-21,1626245,Cristiano Felicio,1610612741,Chicago Bulls,17.950000,0.400,0.500,5,0,8,1,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26102,2017-18,1627775,Patrick McCaw,1610612744,Golden State Warriors,18.656667,0.667,0.000,4,1,3,1,1,-9
26103,2017-18,202691,Klay Thompson,1610612744,Golden State Warriors,38.150000,0.429,0.000,16,3,6,2,2,0
26104,2017-18,2548,Dwyane Wade,1610612739,Cleveland Cavaliers,28.495000,0.300,1.000,8,3,2,0,2,-1
26105,2017-18,203935,Marcus Smart,1610612738,Boston Celtics,35.050000,0.313,0.667,12,3,9,2,2,-8


### Cleaning boxstats

### Saving Data <br>
Now we save the data we gathered we requested from the NBA API into csv files

In [ ]:
# NEED TO RUN BOXSTATS REQUEST BLOCK FIRST

# Restrict boxscfinder to teams the random teams we selected
# boxscfinder = boxscfinder[boxscfinder['TEAM_ID'].isin(team_id_random)]

# Restrict to the player stats we want
# boxscfinder = boxscfinder[['SEASON_YEAR', 'PLAYER_ID', 'PLAYER_NAME', 'MIN', 'FG_PCT', 'FT_PCT', 'PTS', 'AST', 'REB', 'STL', 'BLK', 'PLUS_MINUS']
# boxscfinder

In [11]:
#Saving the Data to a csv file
logsDf.to_csv("logPlayerStats.csv")

# NEED TO RUN BOXSTATS REQUEST BLOCK FIRST
# boxscfinder.to_csv("boxscfinder.csv")

### Manipulating/Plotting CSV File Data

In [12]:
import seaborn as sns

/nrnb/opt/anaconda3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/nrnb/opt/anaconda3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/nrnb/opt/anaconda3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/nrnb/opt/anaconda3.7/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [14]:
#Need to plot the distributions data using either seaborn or matplotlib

dataDf = pd.read_csv("logPlayerStats.csv")
#dataDf['TIMES_TRADED'] = np.nan
pid_lst = list(player_ids[0])
pid_lst[0]

season = ['2016-17','2017-18','2018-19','2019-20','2020-21']

playerStats = pd.DataFrame()
for pid in pid_lst:
    time.sleep(1)
    career = playercareerstats.PlayerCareerStats(player_id=pid)
    temp = career.get_data_frames()[0]
    temp = temp[temp['SEASON_ID'].isin(season)]
    playerStats = playerStats.append(temp)
    

In [15]:
season = ['2016-17','2017-18','2018-19','2019-20','2020-21']
for s in season:
    playerStats["Traded " + s] = 0
    
for pid in pid_lst:
    for s in season:
        tempDf = dataDf.loc[(dataDf['PLAYER_ID'] == pid) & (dataDf['SEASON_YEAR'] == s)]
        tempDf = tempDf.reset_index(drop=True)
        tempDf = tempDf.drop_duplicates(subset=['TEAM_ID'])
        traded = len(tempDf.index)-1
        
        if traded >= 0:
            playerStats.loc[(playerStats['PLAYER_ID'] == pid), ["Traded " + s]] = traded
        else:
            playerStats.loc[(playerStats['PLAYER_ID'] == pid), ["Traded " + s]] = 0

In [16]:
playerStats = playerStats.reset_index(drop=True)
playerStats

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,STL,BLK,TOV,PF,PTS,Traded 2016-17,Traded 2017-18,Traded 2018-19,Traded 2019-20,Traded 2020-21
0,1630208,2020-21,00,1610612766,CHA,23.0,18,0,63.0,4,...,0,0,3,7,15,0,0,0,0,0
1,202714,2016-17,00,1610612762,UTA,27.0,55,9,1205.0,170,...,42,3,90,100,430,0,0,1,0,0
2,202714,2017-18,00,1610612753,ORL,28.0,69,3,1365.0,178,...,54,8,89,91,473,0,0,1,0,0
3,202714,2018-19,00,1610612763,MEM,29.0,53,3,1204.0,161,...,45,4,62,81,421,0,0,1,0,0
4,202714,2018-19,00,1610612766,CHA,29.0,4,0,42.0,2,...,2,0,4,3,9,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,1630203,2020-21,00,1610612766,CHA,24.0,7,0,27.0,8,...,1,0,1,0,18,0,0,0,0,0
3214,1630204,2020-21,00,1610612750,MIN,21.0,2,0,4.0,0,...,0,0,1,0,0,0,0,0,0,0
3215,1630205,2020-21,00,1610612739,CLE,23.0,40,0,499.0,62,...,17,13,23,53,165,0,0,0,0,0
3216,1630206,2020-21,00,1610612746,LAC,20.0,4,1,84.0,14,...,4,0,7,5,35,0,0,0,0,0


In [17]:
playerStats.to_csv("PlayerStats.csv")

In [18]:
playerStats

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,STL,BLK,TOV,PF,PTS,Traded 2016-17,Traded 2017-18,Traded 2018-19,Traded 2019-20,Traded 2020-21
0,1630208,2020-21,00,1610612766,CHA,23.0,18,0,63.0,4,...,0,0,3,7,15,0,0,0,0,0
1,202714,2016-17,00,1610612762,UTA,27.0,55,9,1205.0,170,...,42,3,90,100,430,0,0,1,0,0
2,202714,2017-18,00,1610612753,ORL,28.0,69,3,1365.0,178,...,54,8,89,91,473,0,0,1,0,0
3,202714,2018-19,00,1610612763,MEM,29.0,53,3,1204.0,161,...,45,4,62,81,421,0,0,1,0,0
4,202714,2018-19,00,1610612766,CHA,29.0,4,0,42.0,2,...,2,0,4,3,9,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,1630203,2020-21,00,1610612766,CHA,24.0,7,0,27.0,8,...,1,0,1,0,18,0,0,0,0,0
3214,1630204,2020-21,00,1610612750,MIN,21.0,2,0,4.0,0,...,0,0,1,0,0,0,0,0,0,0
3215,1630205,2020-21,00,1610612739,CLE,23.0,40,0,499.0,62,...,17,13,23,53,165,0,0,0,0,0
3216,1630206,2020-21,00,1610612746,LAC,20.0,4,1,84.0,14,...,4,0,7,5,35,0,0,0,0,0


In [19]:
playerStats[playerStats['PLAYER_ID'] ==1626144 ]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,STL,BLK,TOV,PF,PTS,Traded 2016-17,Traded 2017-18,Traded 2018-19,Traded 2019-20,Traded 2020-21
60,1626144,2016-17,00,1610612743,DEN,21.0,55,41,1406.0,208,...,41,13,123,95,603,0,1,0,0,0
61,1626144,2017-18,00,1610612743,DEN,22.0,42,0,752.0,129,...,21,4,72,47,359,0,1,0,0,0
62,1626144,2017-18,00,1610612752,NYK,22.0,22,14,493.0,75,...,20,7,41,33,194,0,1,0,0,0
63,1626144,2017-18,00,0,TOT,22.0,64,14,1244.0,204,...,41,11,113,80,553,0,1,0,0,0
64,1626144,2018-19,00,1610612752,NYK,23.0,59,42,1607.0,330,...,43,19,140,103,873,0,1,0,0,0
65,1626144,2019-20,00,1610612762,UTA,24.0,54,2,850.0,151,...,24,10,76,72,395,0,1,0,0,0


In [20]:
season_stats = pd.DataFrame()
for pid in pid_lst:
    time.sleep(1)
    career = playercareerstats.PlayerCareerStats(player_id=pid)
    temp = career.career_totals_regular_season.get_data_frame()
    season_stats = season_stats.append(temp)  

In [21]:
season_stats

,PLAYER_ID,LEAGUE_ID,Team_ID,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,1630208,00,0,59,5,406.0,59,90,0.655555,0,...,0.711538,34,57,91,15,9,19,24,56,155
0,202714,00,0,456,56,8393.0,1165,2752,0.423328,302,...,0.731462,118,776,894,1421,300,24,560,568,2997
0,1630210,00,0,60,1,341.0,67,180,0.372222,41,...,0.869565,1,26,27,24,11,1,24,34,195
0,1630211,00,0,10,0,88.0,3,13,0.230769,1,...,0.800000,1,13,14,4,0,2,3,10,11
0,1630214,00,0,98,12,1594.0,236,452,0.522123,29,...,0.643564,120,252,372,115,85,47,71,165,566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1630203,00,0,7,0,27.0,8,12,0.666666,2,...,0.000000,0,1,1,3,1,0,1,0,18
0,1630204,00,0,2,0,4.0,0,0,0.000000,0,...,0.000000,0,0,0,0,0,0,1,0,0
0,1630205,00,0,82,8,1092.0,146,313,0.466453,17,...,0.681318,53,134,187,50,37,22,45,113,371
0,1630206,00,0,22,1,206.0,32,82,0.390243,8,...,0.800000,4,26,30,9,8,3,14,14,84


In [22]:
season = ['2016-17','2017-18','2018-19','2019-20','2020-21']
for s in season:
    season_stats["Traded " + s] = 0
    
for pid in pid_lst:
    for s in season:
        tempDf = dataDf.loc[(dataDf['PLAYER_ID'] == pid) & (dataDf['SEASON_YEAR'] == s)]
        tempDf = tempDf.reset_index(drop=True)
        tempDf = tempDf.drop_duplicates(subset=['TEAM_ID'])
        traded = len(tempDf.index)-1
        
        if traded >= 0:
            season_stats.loc[(season_stats['PLAYER_ID'] == pid), ["Traded " + s]] = traded

        else:
            season_stats.loc[(playerStats['PLAYER_ID'] == pid), ["Traded " + s]] = 0

In [23]:
career_stats

NameError: name 'career_stats' is not defined

In [ ]:
carrer_stats[season_stats['Traded 2017-18'] > 0]

In [ ]:
career_stats.to_csv("careerStats.csv")